In [12]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import OpenAI
from langchain_pinecone import PineconeVectorStore as Pinecone
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import os

In [2]:
loader = PyPDFDirectoryLoader("pdfs")

In [3]:
data = loader.load()

In [4]:
data[0]

Document(metadata={'source': 'pdfs\\NIPS-2017-attention-is-all-you-need-Paper.pdf', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and convolutions\nentirely. Experiments on two machine translation tasks show

In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                chunk_overlap=20
)

In [6]:
text_chunks = text_splitter.split_documents(data)

In [7]:
print(text_chunks[75].page_content)

[31] Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc V Le, Mohammad Norouzi, Wolfgang
Macherey, Maxim Krikun, Yuan Cao, Qin Gao, Klaus Macherey, et al. Google’s neural machine
translation system: Bridging the gap between human and machine translation. arXiv preprint
arXiv:1609.08144 , 2016.
[32] Jie Zhou, Ying Cao, Xuguang Wang, Peng Li, and Wei Xu. Deep recurrent models with
fast-forward connections for neural machine translation. CoRR , abs/1606.04199, 2016.
11


In [8]:
len(text_chunks)

76

In [10]:
embedding = OpenAIEmbeddings(api_key=my_key)
embedding.embed_query('My name is abraham')

C:\Users\mayuo\anaconda3\envs\genAI\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


[-0.028025625649539605,
 -0.008576044357860333,
 -0.009126683212956243,
 -0.023772413512634254,
 -0.032304154857970294,
 0.028810443690954795,
 -0.01879767601312706,
 -0.00399055031379754,
 -0.00798110062759508,
 -0.0032658586874406096,
 -0.00567727785243261,
 0.00781654152532543,
 0.005259551760199677,
 0.001640840928571973,
 -0.0006989792084196398,
 -0.01579764215292051,
 0.03174718673499305,
 -0.015152066142249631,
 0.01992427079484422,
 0.009955806129540775,
 0.02354456359419631,
 -0.009816564098796464,
 0.01989895372331888,
 -0.0039114353966033825,
 -0.01448117212873089,
 0.008063380178729904,
 0.008386168184065344,
 -0.013063434749762438,
 0.02227872678173484,
 -0.04329161488836316,
 0.010525432788280688,
 -0.0087849074039768,
 0.0108798671330228,
 -0.033240871603247416,
 -0.011620381230591174,
 -0.016354610275897755,
 0.0037848519016217405,
 -0.023949631150666575,
 0.015088773463436286,
 0.013215335316269419,
 0.018924259508108703,
 -0.0033987716831342172,
 -0.003636115969055427

In [11]:
len(embedding.embed_query('My name is abraham'))

1536

In [17]:
from pinecone import Pinecone as pc

In [19]:
pine = pc(api_key= PINECONE_API_KEY)

### Creating Embeddings for each Text Chuck

In [28]:
from langchain_pinecone import PineconeVectorStore

In [35]:
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [42]:
docsearch = Pinecone.from_texts(
    texts=[t.page_content for t in text_chunks],
    embedding=embedding,
    index_name=index_name
    #pinecone_api_key=PINECONE_API_KEY
)

In [43]:
docsearch

In [44]:
docsearch.as_retriever()

VectorStoreRetriever(tags=['Pinecone', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.pinecone.Pinecone object at 0x00000250EC325970>)

In [46]:
query = "attention is all you need for transformers"

In [47]:
doc = docsearch.similarity_search(query)

In [48]:
doc

[Document(page_content='the input or output sequences [ 2,16]. In all but a few cases [ 22], however, such attention mechanisms\nare used in conjunction with a recurrent network.\nIn this work we propose the Transformer, a model architecture eschewing recurrence and instead\nrelying entirely on an attention mechanism to draw global dependencies between input and output.\nThe Transformer allows for signiﬁcantly more parallelization and can reach a new state of the art in'),
 Document(page_content='the input or output sequences [ 2,16]. In all but a few cases [ 22], however, such attention mechanisms\nare used in conjunction with a recurrent network.\nIn this work we propose the Transformer, a model architecture eschewing recurrence and instead\nrelying entirely on an attention mechanism to draw global dependencies between input and output.\nThe Transformer allows for signiﬁcantly more parallelization and can reach a new state of the art in'),
 Document(page_content='7 Conclusion\nIn thi

In [50]:
llm = OpenAI(openai_api_key = my_key)

In [52]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type='stuff', retriever = docsearch.as_retriever())

In [54]:
query = "How many attention heads where used"
qa.run(query)

' 8'

In [56]:
import sys
while True:
    user_input = input(f'Input Prompt: ')
    if user_input == 'exit':
        print('Exit')
        sys.exit()
    if user_input == '':
        continue
    result = qa({'query': user_input})
    print(f"Answer: {result['result']}")

Input Prompt:  What dimentions was using in this paper


C:\Users\mayuo\anaconda3\envs\genAI\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Answer:  The dimensions used in this paper are the representation dimensionality d and the position dimension i, which is used for the positional encoding.


Input Prompt:  exit


Exit


SystemExit: 

C:\Users\mayuo\anaconda3\envs\genAI\lib\site-packages\IPython\core\interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
